In [2]:
from hmm import SUTDHMM
import os

### Part 2

Estimate the emission parameters from the training set using MLE

In [3]:
from hmm import SUTDHMM
languages = ['EN', 'SG', 'CN', 'FR']

for l in languages:
    model = SUTDHMM()
    model.train(input_filename='./{}/train'.format(l))
    print("Finish training for {}".format(l))

    with open("./{}/dev.in".format(l)) as in_file, open("./{}/dev.p2.out".format(l), 'w+') as out_file:
        for line in in_file:
            word = line.strip()
            if (word == ''):
                out_file.write("\n")
            else:
                out_file.write("{} {}\n".format(word, model.predict_label_using_emission(word)))
    print("Finished: {}".format(l))
    
    output = os.popen("python3 EvalScript/evalResult.py {0}/dev.out {0}/dev.p2.out".format(l)).read()
    print("Language: {}".format(l))
    print(output)
    print("----------------------")

Finish training for EN
Finished: EN
Language: EN

#Entity in gold data: 226
#Entity in prediction: 706

#Correct Entity : 133
Entity  precision: 0.1884
Entity  recall: 0.5885
Entity  F: 0.2854

#Correct Sentiment : 45
Sentiment  precision: 0.0637
Sentiment  recall: 0.1991
Sentiment  F: 0.0966

----------------------
Finish training for SG
Finished: SG
Language: SG

#Entity in gold data: 1382
#Entity in prediction: 2764

#Correct Entity : 511
Entity  precision: 0.1849
Entity  recall: 0.3698
Entity  F: 0.2465

#Correct Sentiment : 272
Sentiment  precision: 0.0984
Sentiment  recall: 0.1968
Sentiment  F: 0.1312

----------------------
Finish training for CN
Finished: CN
Language: CN

#Entity in gold data: 362
#Entity in prediction: 1688

#Correct Entity : 114
Entity  precision: 0.0675
Entity  recall: 0.3149
Entity  F: 0.1112

#Correct Sentiment : 71
Sentiment  precision: 0.0421
Sentiment  recall: 0.1961
Sentiment  F: 0.0693

----------------------
Finish training for FR
Finished: FR
Langua

In [11]:
languages = ['EN', 'SG', 'CN', 'FR']

for l in languages:
    model = SUTDHMM()
    model.train(input_filename='./{}/train'.format(l))

    print("Finish training for {}".format(l))

    print("----------Viterbi for {0}------------".format(l))
    with open("./{}/dev.in".format(l)) as in_file, open("./{}/dev.p3.out".format(l), 'w+') as out_file:
        read_data = in_file.read()
        sentences = list(filter(lambda x: len(x) > 0, read_data.split('\n\n')))
        sentences = list(map(lambda x: ' '.join(x.split('\n')), sentences))
        for sentence in sentences:
            sentence_labels, chance = model.viterbi(sentence)
            for idx, word in enumerate(sentence.split()):
                out_file.write("{} {}\n".format(word, sentence_labels[idx]))
            out_file.write('\n')
        out_file.close()
        in_file.close()

    print("Viterbi Finished: {}".format(l))
    
    output = os.popen(
        "python3 EvalScript/evalResult.py {0}/dev.out {0}/dev.p3.out".format(l)).read()
    print("Language: {}".format(l))
    print(output)

Finish training for EN
----------Viterbi for EN------------
Viterbi Finished: EN
Language: EN

#Entity in gold data: 226
#Entity in prediction: 116

#Correct Entity : 74
Entity  precision: 0.6379
Entity  recall: 0.3274
Entity  F: 0.4327

#Correct Sentiment : 51
Sentiment  precision: 0.4397
Sentiment  recall: 0.2257
Sentiment  F: 0.2982

Finish training for SG
----------Viterbi for SG------------
Viterbi Finished: SG
Language: SG

#Entity in gold data: 1382
#Entity in prediction: 499

#Correct Entity : 228
Entity  precision: 0.4569
Entity  recall: 0.1650
Entity  F: 0.2424

#Correct Sentiment : 152
Sentiment  precision: 0.3046
Sentiment  recall: 0.1100
Sentiment  F: 0.1616

Finish training for CN
----------Viterbi for CN------------
Viterbi Finished: CN
Language: CN

#Entity in gold data: 362
#Entity in prediction: 173

#Correct Entity : 28
Entity  precision: 0.1618
Entity  recall: 0.0773
Entity  F: 0.1047

#Correct Sentiment : 17
Sentiment  precision: 0.0983
Sentiment  recall: 0.0470
Se

In [13]:
languages = ['EN', 'SG', 'CN', 'FR']

for l in languages:
    model = SUTDHMM(standardise_word=True)
    model.train(input_filename='./{}/train'.format(l))

    print("Finish training for {}".format(l))

    print("----------Viterbi for {0}------------".format(l))
    with open("./{}/dev.in".format(l)) as in_file, open("./{}/dev.p3.out".format(l), 'w+') as out_file:
        read_data = in_file.read()
        sentences = list(filter(lambda x: len(x) > 0, read_data.split('\n\n')))
        sentences = list(map(lambda x: ' '.join(x.split('\n')), sentences))
        for sentence in sentences:
            sentence_labels, chance = model.viterbi(sentence)
            for idx, word in enumerate(sentence.split()):
                out_file.write("{} {}\n".format(word, sentence_labels[idx]))
            out_file.write('\n')
        out_file.close()
        in_file.close()

    print("Improved Viterbi Finished: {}".format(l))
    
    output = os.popen(
        "python3 EvalScript/evalResult.py {0}/dev.out {0}/dev.p3.out".format(l)).read()
    print("Language: {}".format(l))
    print(output)

Finish training for EN
----------Viterbi for EN------------
Improved Viterbi Finished: EN
Language: EN

#Entity in gold data: 226
#Entity in prediction: 98

#Correct Entity : 62
Entity  precision: 0.6327
Entity  recall: 0.2743
Entity  F: 0.3827

#Correct Sentiment : 41
Sentiment  precision: 0.4184
Sentiment  recall: 0.1814
Sentiment  F: 0.2531



KeyboardInterrupt: 